In [ ]:

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini",
                   api_key=input("Enter your OpenAI API key: "))

In [56]:
model.invoke("What's on the front page of hacker news?").content

"I don't have real-time access to the internet or current data, so I can't provide you with the latest content from Hacker News or any other website. However, you can easily check the front page of Hacker News by visiting their website at [news.ycombinator.com](https://news.ycombinator.com). If you have any specific topics or articles in mind, feel free to ask!"

In [57]:
from langchain_core.tools import tool
import requests

@tool
def read_webpage(url: str) -> str:
    """
    Fetch a url's content and return it as a string
    """
    print("Fetching content from", url)
    response = requests.get(url)
    
    return response.text


tools = [read_webpage]

In [58]:
tool_model = model.bind_tools(tools)

In [59]:
tool_model.invoke("what's on the front page of hacker news?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_gu9a4Rj4PXXEoGp0b3dvXdDi', 'function': {'arguments': '{"url":"https://news.ycombinator.com/"}', 'name': 'read_webpage'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 60, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f2cd28694a', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-78c65fd4-10d8-49fb-ae0a-b126cdc88eee-0', tool_calls=[{'name': 'read_webpage', 'args': {'url': 'https://news.ycombinator.com/'}, 'id': 'call_gu9a4Rj4PXXEoGp0b3dvXdDi', 'type': 'tool_call'}], usage_metadata={'input_tokens': 60, 'output_tokens': 24, 'total_tokens': 84, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'outp

In [60]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import END, START, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode

system_prompt = """
    You must provide your answers in a format readable in a terminal. Do not provide any text other than the answer.
    Your responses must not exceed 120 characters in width but can be multiple lines.
"""


def call_agent(state: MessagesState):
    response = tool_model.invoke([
        ("system", system_prompt),
        *state["messages"]
    ])

    return {
        "messages": [response]
    }


def after_prompt_edge(state: MessagesState):
    if state["messages"][-1].tool_calls:
        return "tools"
    return END


tool_node = ToolNode(tools)

workflow = StateGraph(MessagesState)

workflow.add_node("prompt", call_agent)

workflow.add_node("tools", tool_node)

workflow.add_edge(START, "prompt")
workflow.add_edge("tools", "prompt")
workflow.add_conditional_edges("prompt", after_prompt_edge)

app = workflow.compile()

In [61]:
messages = [
    HumanMessage(content="Return the top 10 stories on hacker news, their titles, URLs, and comment URLs.")
]

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print(final_state['messages'][-1].content)

Fetching content from https://news.ycombinator.com/
1. [28h Days: year 1 update](https://sidhion.com/blog/28h_days_update_1/) - [Comments](https://news.ycombinator.com/item?id=42639779)  
2. [NeuralSVG: An Implicit Representation for Text-to-Vector Generation](https://sagipolaczek.github.io/NeuralSVG/) - [Comments](https://news.ycombinator.com/item?id=42636873)  
3. [Orca that carried dead calf for weeks appears to be in mourning again](https://www.nytimes.com/2025/01/02/us/mother-orca-tahlequah-calf-dead.html) - [Comments](https://news.ycombinator.com/item?id=42599522)  
4. [Cracking a 512-bit DKIM key for less than $8 in the cloud](https://dmarcchecker.app/articles/crack-512-bit-dkim-rsa-key) - [Comments](https://news.ycombinator.com/item?id=42633501)  
5. [Show HN: Atlas of Space](https://atlasof.space/) - [Comments](https://news.ycombinator.com/item?id=42634787)  
6. [Show HN: Stagehand – an open source browser automation framework powered by AI](https://github.com/browserbase/stag

In [63]:

question = input("What is your follow-up question?")
messages.append(HumanMessage(content=question))

final_state = app.invoke(
    {"messages": messages},
)

messages.append(final_state['messages'][-1])

print("You asked:", question)
print(final_state['messages'][-1].content)

Fetching content from https://operating-system-in-1000-lines.vercel.app/en
You asked: read and summarize the os article
The article introduces building a basic operating system in 1,000 lines of code. It covers:

- Simple OS functions and kernel basics.
- Implementation of context switching, paging, user mode, a shell, and disk I/O.
- Debugging challenges unique to OS development.
- Requires familiarity with C and UNIX-like systems.
- Implementation examples available on GitHub.

The goal is to make OS development accessible and enjoyable.


In [67]:
state = app.invoke({"messages": [
	HumanMessage(content="What are the recent blog posts posted by Source Allies?"),
]})

print(state['messages'][-1].content)

Fetching content from https://sourceallies.com/blog/
1. [Better Nightly Emails through Lambda, SES, and React](https://sourceallies.com/2024/10/better-nightly-emails/)
2. [Some fun JavaScript features for API functionality](https://sourceallies.com/2024/10/fun-javascript-features/)
3. [The Road to Mordor: A Software Engineer’s Guide to Delivering Complex Features](https://sourceallies.com/2024/09/the-road-to-mordor/)
4. [What Is A Delivery Lead?](https://sourceallies.com/2024/08/what-is-a-delivery-lead/)
5. [Navigating Agentic AI Reasoning: ReAct vs ReWOO](https://sourceallies.com/2024/08/react-vs-rewoo/)
